# Texturepack

Helper methods to streamline creating minecraft texture pack for segmentation

In [9]:
import os
from pathlib import Path
from glob import glob
from PIL import Image
import cv2
import numpy as np

from mapping import mapping, mapping_old

In [10]:
TEXTURES_PATH = Path(r'C:\\Users\\Lukasz\AppData\\Roaming\\.mcRPW\\projects\\Segmentation\\project_files\\assets\\minecraft\\textures')
TRANSPARENT = (0, 0, 0, 0)

In [12]:
def replace_image_with_rgb(path, color):
    with Image.open(path) as image:
        new_image = Image.new('RGBA', image.size, color)
    os.remove(path)
    new_image.save(path)

In [4]:
def replace_image_with_hsv(path, hsv_color):
    input_image = cv2.imread(str(path))
    output_image = np.full_like(input_image, np.array(hsv_color, dtype=np.uint8), dtype=np.uint8)
    output_image = cv2.cvtColor(output_image, cv2.COLOR_HSV2BGR)
    cv2.imwrite(str(path), output_image)

In [16]:
path = TEXTURES_PATH / 'block' / 'grass.png'
replace_image_with_hsv(path, mapping['grass'])

In [9]:
paths = list(glob(str(TEXTURES_PATH / 'block') + '/*.png'))
for path in paths:
    if 'leaves' in path:
        print(path)
        replace_image_with_hsv(path, (0, 0, 0))


C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\acacia_leaves.png
C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\azalea_leaves.png
C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\bamboo_large_leaves.png
C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\bamboo_small_leaves.png
C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\birch_leaves.png
C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\cherry_leaves.png
C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\dark_oak_leaves.png
C:\Users\Lukasz\AppData\Roaming\.mcRPW\projects\Segmentation\project_files\assets\minecraft\textures\block\flowe

# Unique colors

In [5]:
paths = list(glob(str(TEXTURES_PATH) + '/**/*.png'))
colors_rgb = []
for path in paths:
    input_image = cv2.imread(path)
    color_rgb = tuple(input_image[0, 0, ::-1])
    colors_rgb.append(color_rgb)
    
colors_rgb = list(set(colors_rgb))
colors_hsv = cv2.cvtColor(np.array(colors_rgb, dtype=np.uint8).reshape(1, -1, 3), cv2.COLOR_RGB2HLS).reshape(-1, 3)

for rgb, hsv in zip(colors_rgb, colors_hsv):
    print(f'{str(rgb):20} {str(hsv):20}')

(127, 127, 127)      [  0 127   0]       
(0, 0, 0)            [0 0 0]             
(255, 0, 0)          [  0 128 255]       
(255, 255, 255)      [  0 255   0]       
(247, 26, 230)       [152 137 238]       
(255, 201, 14)       [ 23 134 255]       
(128, 255, 128)      [ 60 192 255]       
(34, 127, 0)         [ 52  64 255]       
(0, 0, 255)          [120 128 255]       
(139, 69, 19)        [ 12  79 194]       


## RGB to HSV

In [6]:
def get_label_name(path, mapping):
    input_image = cv2.imread(path)
    color = tuple(input_image[0, 0, ::-1])
    
    if color not in list(mapping.values()):
        return None
    else:
        return [key for key, value in mapping.items() if value == color][0]

In [5]:
replace_image_with_hsv('C:\\Users\\Lukasz\\Downloads\\cat.png', (0, 0, 31))

In [ ]:
get_label_name(str(TEXTURES_PATH / 'block' / 'grass.png'), mapping_old)

In [8]:
# paths = list(glob(str(TEXTURES_PATH / 'block') + '/*.png'))
paths = list(glob(str(TEXTURES_PATH / 'entity') + '/**/*.png'))
for path in paths:
    label_name = get_label_name(path, mapping_old)
    if label_name is None:
        print('Ignoring', path)
        continue
    hsv_color = mapping[label_name]
    replace_image_with_hsv(path, hsv_color)
    # print('changing', path)